# Notebook para processamento dos dados de Comida di Buteco

## Imports

In [1]:
! pip install pandas unidecode

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from unidecode import unidecode

## Cross Matching

In [ ]:
# Leitura dos dados
data = pd.read_csv("../data/dados_filtrados.csv")
cdb = pd.read_csv("bares_cdb2025.csv")

# Normalização dos textos
data["NOME_FANTASIA"] = data["NOME_FANTASIA"].str.upper().str.strip()
data["ENDERECO_COMPLETO"] = data["ENDERECO_COMPLETO"].str.upper().str.strip()

cdb["Nome"] = cdb["Nome"].str.upper().str.strip()
cdb["Endereço"] = cdb["Endereço"].str.upper().str.strip()

# Conjuntos de referência
nomes_ref = set(data["NOME_FANTASIA"])
enderecos_ref = set(data["ENDERECO_COMPLETO"])

# Listas para armazenar resultados
nao_encontrados = []
encontrados = []

# Verificação de existência
for _, row in cdb.iterrows():
    nome = row["Nome"]
    endereco = row["Endereço"]

    if endereco in enderecos_ref:
        encontrados.append({
            "Nome": nome,
            "Endereço": endereco
        })
    else:
        nao_encontrados.append({
            "Nome": nome,
            "Endereço": endereco
        })

# Impressão dos não encontrados
for item in nao_encontrados:
    print(f'Não encontrado: {item["Nome"]} - {item["Endereço"]}')

# Salvando os resultados
pd.DataFrame(nao_encontrados).to_csv("nao_encontrados.csv", index=False, encoding="utf-8-sig")
pd.DataFrame(encontrados).to_csv("encontrados.csv", index=False, encoding="utf-8-sig")

## Fuzzy Matching

In [ ]:
def normalizar(texto):
    if pd.isna(texto):
        return ""
    return unidecode(texto.strip().upper())

def extrair_rua(endereco):
    return normalizar(endereco.split(",")[0])

def nome_bate(nome_csv1, nome_csv2, nome_fantasia):
    palavras = [normalizar(p) for p in nome_csv1.split()]
    nome_csv2 = normalizar(nome_csv2)
    nome_fantasia = normalizar(nome_fantasia)
    return any(p in nome_csv2 or p in nome_fantasia for p in palavras)

def endereco_bate(end1, end2):
    return extrair_rua(end1) in normalizar(end2)

csv1 = pd.read_csv("nao_encontrados.csv") 
csv2 = pd.read_csv("dados_filtrados.csv")  

with open("matchings.txt", "w", encoding="utf-8") as output:
    for _, row1 in csv1.iterrows():
        nome1 = str(row1["Nome"])
        endereco1 = str(row1["Endereço"])

        output.write(f'{nome1},"{endereco1}"\n')
        output.write("matchings (pode haver mais de um):\n")

        achou = False
        for _, row2 in csv2.iterrows():
            nome_real = str(row2.get("NOME", ""))
            nome_fantasia = str(row2.get("NOME_FANTASIA", ""))
            endereco2 = str(row2.get("ENDERECO_COMPLETO", ""))

            if nome_bate(nome1, nome_real, nome_fantasia) and endereco_bate(endereco1, endereco2):
                output.write(f'"{endereco2}"{nome_fantasia, nome_real}\n')
                achou = True

        if not achou:
            output.write("Nenhum matching encontrado.\n")
        output.write("\n")


### Resultado do `matching.txt`

**CANTINHO DA BAIANA**  
Endereço: "AVENIDA ITAITE, 422, SÃO GERALDO, BELO HORIZONTE, MG, BRASIL"  
Matchings:  
"AVENIDA ITAITE, 422, SAO GERALDO, BELO HORIZONTE, MG, BRASIL"  
*('Estabelecimento sem nome', 'CANTINHO DA BAIANA COMERCIAL LTDA')*

**BAR DA LU**  
Endereço: "RUA GERALDA MARINHO, 41, SÃO JOÃO BATISTA, BELO HORIZONTE, MG, BRASIL"  
Matchings:  
"RUA GERALDA MARINHO, 41, SAO JOAO BATISTA, BELO HORIZONTE, MG, BRASIL"  
*('BAR E RESTAURANTE DA LU', '38.029.984 LUZMARINA PEREIRA DAS NEVES')*

---


## Organização para o dataset final

In [ ]:
# === 1. Carregar e normalizar os dados ===
data = pd.read_csv("../data/dados_filtrados.csv")
cdb = pd.read_csv("bares_cdb2025.csv")

data["NOME_FANTASIA"] = data["NOME_FANTASIA"].str.upper().str.strip()
data["ENDERECO_COMPLETO"] = data["ENDERECO_COMPLETO"].str.upper().str.strip()
cdb["Nome"] = cdb["Nome"].str.upper().str.strip()
cdb["Endereço"] = cdb["Endereço"].str.upper().str.strip()

# === 2. Identificar endereços coincidentes ===
enderecos_ref = set(data["ENDERECO_COMPLETO"])
encontrados = cdb[cdb["Endereço"].isin(enderecos_ref)].drop_duplicates(subset=["Endereço"])
encontrados = encontrados.reset_index(drop=True)
encontrados["CDB_ID"] = encontrados.index + 1

# === 3. Criar mapeamentos e atualizar `data` com CDB_ID e nome do CDB ===
map_endereco_to_id = dict(zip(encontrados["Endereço"], encontrados["CDB_ID"]))
map_endereco_to_nome = dict(zip(encontrados["Endereço"], encontrados["Nome"]))

data["CDB"] = data["ENDERECO_COMPLETO"].map(map_endereco_to_id).fillna(0).astype(int)
data["NOME_FANTASIA"] = data.apply(
    lambda row: map_endereco_to_nome.get(row["ENDERECO_COMPLETO"], row["NOME_FANTASIA"]),
    axis=1
)

# === 4. Salvar arquivos atualizados ===
# Ordenar colunas para o CSV de encontrados
cols = ["CDB_ID", "Nome", "Endereço"]
encontrados = encontrados[cols]
encontrados.to_csv("encontrados.csv", index=False, encoding="utf-8-sig")
data.to_csv("dados_com_cdb.csv", index=False, encoding="utf-8-sig")

# === 5. Incluir bares não encontrados ===
try:
    novos_bares = pd.read_csv("cdb_notfound.csv")
    novos_bares.rename(columns={
        "CDB_ID": "CDB",
        "Nome": "NOME",
        "Endereço": "ENDERECO_COMPLETO"
    }, inplace=True)

    # Adicionar colunas padrão
    for col, val in {
        "ID_ATIV_ECON_ESTABELECIMENTO": "indisponivel",
        "CNAE_PRINCIPAL": "indisponivel",
        "DATA_INICIO_ATIVIDADE": "indisponivel",
        "IND_POSSUI_ALVARA": "SIM",
        "NOME_FANTASIA": "indisponivel",
        "GEOMETRIA": "indisponivel"
    }.items():
        novos_bares[col] = val

    # Reordenar colunas
    colunas_ordenadas = [
        "ID_ATIV_ECON_ESTABELECIMENTO", "CNAE_PRINCIPAL", "DATA_INICIO_ATIVIDADE",
        "IND_POSSUI_ALVARA", "ENDERECO_COMPLETO", "NOME", "NOME_FANTASIA", "GEOMETRIA", "CDB"
    ]
    novos_bares = novos_bares[colunas_ordenadas]

    # Concatenar e salvar
    dados_atualizados = pd.concat([data, novos_bares], ignore_index=True)
    dados_atualizados.to_csv("dados_com_cdb.csv", index=False, encoding="utf-8-sig")
except FileNotFoundError:
    print("Arquivo 'cdb_notfound.csv' não encontrado. Pulando inclusão de novos bares.")


In [4]:
# padronizar data types

df = pd.read_csv(
    "dados_com_cdb.csv",
    dtype={
        "ID_ATIV_ECON_ESTABELECIMENTO": str,
        "CNAE_PRINCIPAL": str,
        "IND_POSSUI_ALVARA": str,
        "ENDERECO_COMPLETO": str,
        "NOME": str,
        "NOME_FANTASIA": str,
        "GEOMETRIA": str,
        "CDB": "Int64",  
    },
    parse_dates=["DATA_INICIO_ATIVIDADE"],  # converte para datetime64
    dayfirst=True  
)
df["DATA_INICIO_ATIVIDADE"] = pd.to_datetime(df["DATA_INICIO_ATIVIDADE"], dayfirst=True, errors="coerce")

# print(df.dtypes)

In [5]:
df1 = pd.read_csv("cdb_data.csv")
df2 =  pd.read_csv("bares_cdb2025.csv")

df2["Nome"] = df2["Nome"].str.strip('"')

df1["Nome_upper"] = df1["Nome"].str.upper()
df2["Nome_upper"] = df2["Nome"].str.upper()

df1["CDB_ID"] = df1["CDB_ID"].apply(lambda x: f"{int(x):03d}")

df_merged = pd.merge(df1, df2, on="Nome_upper", how="left")

df_merged = df_merged.drop(columns=["Nome_upper"])
df_merged.rename(columns={"Nome": "NOME"}, inplace=True)

df_merged["NOME"] = df_merged["Nome_x"]
df_merged["ENDERECO"] = df_merged["Endereço_x"]


df_merged.rename(columns={
    "Imagem": "IMAGEM",
    "Petisco": "PETISCO",
    "Descrição do petisco": "DESCRICAO"
}, inplace=True)

df_final = df_merged[["CDB_ID", "NOME", "ENDERECO", "IMAGEM", "PETISCO", "DESCRICAO"]]

df_final.to_csv("complete_cdb_data.csv", index=False, encoding="utf-8")

df_final.head()

,CDB_ID,NOME,ENDERECO,IMAGEM,PETISCO,DESCRICAO
0,001,ARCOS BAR,"RUA DA BAHIA, 1144, CENTRO, BELO HORIZONTE, MG...",https://s2-g1.glbimg.com/TBJlO2gBA-eoaA_0XK6kM...,Panela da Diversidade,Acem cozido com ervas finas acompanhado com ba...
1,002,AZOUGUE FOGO E BAR,"RUA DO OURO, 835, SERRA, BELO HORIZONTE, MG, B...",https://s2-g1.glbimg.com/4nvx0TTUnKRrUtN5CL_bO...,Lingua da vovó Virinha,Língua de boi assada sobre creme de batata e m...
2,003,BAIÚCA,"RUA PIAUI, 1884, SAVASSI, BELO HORIZONTE, MG, ...",https://s2-g1.glbimg.com/jzIpvnVuWznTE1n1R0us6...,Canelada Suína,"Panturrilha suína, mandioca cozida com queijo ..."
3,004,BAR ESTABELECIMENTO,"RUA MONTE ALEGRE, 160, SERRA, BELO HORIZONTE, ...",https://s2-g1.glbimg.com/XMgVOHr-0_Y9lUNhH3Fr4...,Tipracas,"Pelotas de rabada com angu de agrião e ""tiprac..."
4,005,BAR MANIA MINEIRA,"RUA PARACATU, 1099, SANTO AGOSTINHO, BELO HORI...",https://cdb-static-files.s3.amazonaws.com/wp-c...,EXPLOSÃO DE SABOR,Bolinho de mandioca recheado com queijo


## Conclusão

Após o processo de matching automático e os ajustes manuais realizados, obtivemos um datasets principal:

 **`complete_cdb_data.csv`**

Este arquivo contém as informações completas e padronizadas dos bares participantes do Comida di Buteco. As colunas presentes são:

- : identificador único do bar.
- `NOME`: nome oficial do estabelecimento.
- `ENDERECO`: endereço completo do bar.
- `IMAGEM`: URL da imagem representativa do bar ou do petisco.
- `PETISCO`: nome do petisco inscrito no concurso.
- `DESCRICAO`: descrição detalhada do petisco.

Vamos usar o `CDB_ID` como chave estrangeira para linkar os dados do Comida di Buteco com os dados gerais dos bares e restaurantes.